<a href="https://colab.research.google.com/github/laura8am/portfolio-data-analyst/blob/main/proyecto3_residuos_economia_circular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto 3: De los Datos a las Decisiones
## Análisis de Residuos y Economía Circular

**Autora:** Laura Ochoa M.
**Fecha inicio:** Febrero 2026
**Stack:** Python · SQL · Plotly · Seaborn

### Preguntas de análisis
1. ¿Qué países de la UE tienen las tasas de reciclaje más altas?
2. ¿Cuánto residuo genera México per cápita vs Europa?
3. ¿Dónde están los mayores gaps entre México y el benchmark europeo?
4. ¿Qué oportunidades existen para una empresa gestora de residuos en México?

### Fuentes de datos
- EUROSTAT: Municipal waste statistics (ENV_WASMUN)
- SEMARNAT: Informe de la Situación del Medio Ambiente en México
- BID: Diagnóstico de gestión de residuos en LAC 2022

In [1]:
# ============================================================
# LIBRERÍAS
# ============================================================
# sqlite3 ya viene con Python — no necesita instalación

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sqlite3
import warnings

warnings.filterwarnings('ignore')

# Configuración visual
sns.set_theme(style="whitegrid", palette="viridis")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.family'] = 'sans-serif'

print("✅ Librerías cargadas correctamente")
print(f"   pandas  {pd.__version__}")
print(f"   numpy   {np.__version__}")

✅ Librerías cargadas correctamente
   pandas  2.2.2
   numpy   2.0.2


In [2]:
# ============================================================
# DATASET: GENERACIÓN Y GESTIÓN DE RESIDUOS
# Fuentes: EUROSTAT 2022, SEMARNAT 2021, BID 2022
# ============================================================

data_paises = {
    'pais': [
        # Líderes europeos
        'Alemania', 'Austria', 'Países Bajos', 'Bélgica', 'Dinamarca',
        # Europa media
        'Francia', 'España', 'Italia', 'Portugal', 'Polonia',
        # Europa rezagada (pero mejorando)
        'Rumanía', 'Bulgaria', 'Grecia',
        # América Latina
        'Brasil', 'Colombia', 'Argentina', 'Chile', 'Perú',
        # México
        'México'
    ],
    'region': [
        'Europa', 'Europa', 'Europa', 'Europa', 'Europa',
        'Europa', 'Europa', 'Europa', 'Europa', 'Europa',
        'Europa', 'Europa', 'Europa',
        'América Latina', 'América Latina', 'América Latina',
        'América Latina', 'América Latina',
        'México'
    ],
    'kg_per_capita': [
        # kg de residuos municipales por habitante por año
        632, 560, 527, 505, 844,
        541, 530, 591, 516, 358,
        312, 365, 533,
        379, 270, 359, 395, 218,
        415  # México
    ],
    'tasa_reciclaje_pct': [
        # % del total de residuos que se recicla o composta
        67.3, 57.7, 55.9, 54.3, 46.4,
        44.8, 35.7, 48.7, 28.5, 35.5,
        13.7, 32.1, 20.2,
        4.0, 17.0, 10.5, 11.2, 1.5,
        9.3  # México
    ],
    'tasa_relleno_pct': [
        # % que va a relleno sanitario
        29.5, 30.0, 2.0, 10.3, 2.0,
        25.0, 51.6, 20.9, 63.5, 39.1,
        68.9, 54.2, 76.6,
        52.0, 61.0, 55.0, 59.0, 75.0,
        61.0  # México
    ],
    'tasa_incineracion_pct': [
        # % que va a incineración con recuperación energética
        0.0, 12.0, 42.0, 35.0, 51.0,
        30.0, 12.0, 18.0, 8.0, 22.5,
        1.2, 0.0, 0.0,
        0.1, 0.0, 0.5, 0.0, 0.0,
        0.0  # México
    ],
    'disposicion_inadecuada_pct': [
        # % en tiraderos a cielo abierto o quema
        0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.2, 0.0, 0.0, 0.0,
        16.2, 13.7, 3.2,
        25.0, 21.0, 22.0, 17.0, 32.0,
        21.0  # México
    ],
    'anio': [2022]*19,
    'fuente': [
        'EUROSTAT']*13 + ['BID 2022']*5 + ['SEMARNAT 2021']
}

df = pd.DataFrame(data_paises)

print("✅ Dataset creado")
print(f"   {len(df)} países | {len(df.columns)} variables")
print(f"\n{'='*50}")
print(df[['pais', 'region', 'kg_per_capita', 'tasa_reciclaje_pct']].to_string(index=False))

✅ Dataset creado
   19 países | 9 variables

        pais         region  kg_per_capita  tasa_reciclaje_pct
    Alemania         Europa            632                67.3
     Austria         Europa            560                57.7
Países Bajos         Europa            527                55.9
     Bélgica         Europa            505                54.3
   Dinamarca         Europa            844                46.4
     Francia         Europa            541                44.8
      España         Europa            530                35.7
      Italia         Europa            591                48.7
    Portugal         Europa            516                28.5
     Polonia         Europa            358                35.5
     Rumanía         Europa            312                13.7
    Bulgaria         Europa            365                32.1
      Grecia         Europa            533                20.2
      Brasil América Latina            379                 4.0
    Colomb

In [3]:
# ============================================================
# INTRODUCCIÓN A SQL
# ============================================================
# SQL = Structured Query Language
# Es el lenguaje para hablar con bases de datos.
# Hoy aprendes 3 comandos fundamentales:
#   SELECT  → qué columnas quiero ver
#   FROM    → de qué tabla
#   WHERE   → con qué condición
#   ORDER BY → ordenar resultados
#   LIMIT   → cuántos resultados mostrar
# ============================================================

# Paso 1: Crear una base de datos en memoria
# (como abrir un archivo de Excel, pero en SQL)
conn = sqlite3.connect(':memory:')

# Paso 2: Guardar nuestro DataFrame como tabla SQL
df.to_sql('residuos', conn, index=False, if_exists='replace')

print("✅ Base de datos SQL creada")
print("   Tabla: 'residuos'")
print(f"   Registros: {len(df)}")

✅ Base de datos SQL creada
   Tabla: 'residuos'
   Registros: 19


In [4]:
# ============================================================
# QUERY 1: Ver todos los datos
# ============================================================
# SELECT * significa "dame todas las columnas"
# FROM residuos significa "de la tabla residuos"
# LIMIT 5 significa "solo muéstrame 5 filas"

query_1 = """
SELECT *
FROM residuos
LIMIT 5
"""

resultado_1 = pd.read_sql_query(query_1, conn)
print("QUERY 1: Primeras 5 filas de la tabla")
print("="*50)
print(resultado_1.to_string(index=False))

QUERY 1: Primeras 5 filas de la tabla
        pais region  kg_per_capita  tasa_reciclaje_pct  tasa_relleno_pct  tasa_incineracion_pct  disposicion_inadecuada_pct  anio   fuente
    Alemania Europa            632                67.3              29.5                    0.0                         0.0  2022 EUROSTAT
     Austria Europa            560                57.7              30.0                   12.0                         0.0  2022 EUROSTAT
Países Bajos Europa            527                55.9               2.0                   42.0                         0.0  2022 EUROSTAT
     Bélgica Europa            505                54.3              10.3                   35.0                         0.0  2022 EUROSTAT
   Dinamarca Europa            844                46.4               2.0                   51.0                         0.0  2022 EUROSTAT


In [5]:
# ============================================================
# QUERY 2: ¿Qué países reciclan más del 50%?
# ============================================================
# WHERE filtra filas según una condición
# ORDER BY ordena los resultados
# DESC = descendente (de mayor a menor)

query_2 = """
SELECT pais,
       region,
       tasa_reciclaje_pct
FROM residuos
WHERE tasa_reciclaje_pct > 50
ORDER BY tasa_reciclaje_pct DESC
"""

resultado_2 = pd.read_sql_query(query_2, conn)
print("QUERY 2: Países con tasa de reciclaje > 50%")
print("="*50)
print(resultado_2.to_string(index=False))
print(f"\n→ {len(resultado_2)} países superan el 50% de reciclaje")
print(f"→ México está en {df[df['pais']=='México']['tasa_reciclaje_pct'].values[0]}%")

QUERY 2: Países con tasa de reciclaje > 50%
        pais region  tasa_reciclaje_pct
    Alemania Europa                67.3
     Austria Europa                57.7
Países Bajos Europa                55.9
     Bélgica Europa                54.3

→ 4 países superan el 50% de reciclaje
→ México está en 9.3%


In [6]:
# ============================================================
# QUERY 3: ¿Dónde está México comparado con cada región?
# ============================================================
# GROUP BY agrupa filas por una categoría
# AVG() calcula el promedio
# ROUND() redondea decimales

query_3 = """
SELECT region,
       ROUND(AVG(kg_per_capita), 1)      AS promedio_kg_per_capita,
       ROUND(AVG(tasa_reciclaje_pct), 1) AS promedio_reciclaje_pct,
       ROUND(AVG(disposicion_inadecuada_pct), 1) AS promedio_disposicion_inadecuada
FROM residuos
GROUP BY region
ORDER BY promedio_reciclaje_pct DESC
"""

resultado_3 = pd.read_sql_query(query_3, conn)
print("QUERY 3: Promedios por región")
print("="*50)
print(resultado_3.to_string(index=False))

QUERY 3: Promedios por región
        region  promedio_kg_per_capita  promedio_reciclaje_pct  promedio_disposicion_inadecuada
        Europa                   524.2                    41.6                              2.6
        México                   415.0                     9.3                             21.0
América Latina                   324.2                     8.8                             23.4


In [7]:
# ============================================================
# VISUALIZACIÓN FINAL - VERSIÓN LIMPIA
# ============================================================

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('', '')  # Sin títulos automáticos, los ponemos manual
)

color_map = {
    'Europa': '#2196F3',
    'América Latina': '#FF9800',
    'México': '#4CAF50'
}

# Gráfica izquierda: tasa de reciclaje
df_sorted = df.sort_values('tasa_reciclaje_pct', ascending=True)

fig.add_trace(
    go.Bar(
        y=df_sorted['pais'],
        x=df_sorted['tasa_reciclaje_pct'],
        orientation='h',
        marker_color=[color_map[r] for r in df_sorted['region']],
        text=df_sorted['tasa_reciclaje_pct'].round(1),
        textposition='outside',
        name='Reciclaje %'
    ),
    row=1, col=1
)

# Gráfica derecha: disposición inadecuada
df_sorted2 = df.sort_values('disposicion_inadecuada_pct', ascending=True)

fig.add_trace(
    go.Bar(
        y=df_sorted2['pais'],
        x=df_sorted2['disposicion_inadecuada_pct'],
        orientation='h',
        marker_color=[color_map[r] for r in df_sorted2['region']],
        text=df_sorted2['disposicion_inadecuada_pct'].round(1),
        textposition='outside',
        name='Disposición inadecuada %'
    ),
    row=1, col=2
)

# Layout
fig.update_layout(
    height=900,
    showlegend=False,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(t=100, l=120),
    title=dict(
        text='Gestión de Residuos: Europa vs América Latina vs México (2022)',
        font=dict(size=16),
        y=0.98
    )
)

# Línea de referencia SIN anotación
fig.add_vline(
    x=60, row=1, col=1,
    line_dash="dash",
    line_color="red"
)

# Títulos de subplots y etiqueta meta — todos en posiciones separadas
fig.add_annotation(
    text="Tasa de Reciclaje (%) por País",
    xref="paper", yref="paper",
    x=0.22, y=1.06,
    showarrow=False,
    font=dict(size=13, color="black")
)

fig.add_annotation(
    text="Disposición Inadecuada (%) por País",
    xref="paper", yref="paper",
    x=0.78, y=1.06,
    showarrow=False,
    font=dict(size=13, color="black")
)

fig.add_annotation(
    text="Meta EU 2030: 60%",
    xref="x", yref="paper",
    x=61, y=0.02,
    showarrow=False,
    font=dict(size=11, color="red"),
    xanchor="left"
)

fig.show()

In [8]:
# ============================================================
# EXPORTAR para Notion
# ============================================================

fig.write_html("sesion1_comparativa_residuos.html")
print("✅ Archivo guardado: sesion1_comparativa_residuos.html")
print("   → Descárgalo y súbelo a Notion")
print("\n📊 HALLAZGOS DE LA SESIÓN 1:")
print(f"   • Alemania recicla {df[df['pais']=='Alemania']['tasa_reciclaje_pct'].values[0]}% de sus residuos")
print(f"   • México recicla {df[df['pais']=='México']['tasa_reciclaje_pct'].values[0]}% — gap de {67.3-9.3:.1f} puntos porcentuales")
print(f"   • México tiene {df[df['pais']=='México']['disposicion_inadecuada_pct'].values[0]}% de disposición inadecuada vs 0% en Europa occidental")
print(f"   • Solo 4 países de LAC tienen datos de reciclaje > 15%")

✅ Archivo guardado: sesion1_comparativa_residuos.html
   → Descárgalo y súbelo a Notion

📊 HALLAZGOS DE LA SESIÓN 1:
   • Alemania recicla 67.3% de sus residuos
   • México recicla 9.3% — gap de 58.0 puntos porcentuales
   • México tiene 21.0% de disposición inadecuada vs 0% en Europa occidental
   • Solo 4 países de LAC tienen datos de reciclaje > 15%


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Verificar que México está en los datos
print(df[df['pais'] == 'México'][['pais', 'tasa_reciclaje_pct', 'disposicion_inadecuada_pct']])
print(f"\nTotal de países en el dataset: {len(df)}")

      pais  tasa_reciclaje_pct  disposicion_inadecuada_pct
18  México                 9.3                        21.0

Total de países en el dataset: 19


 ---
## Sesión 2 — Base de Datos SQL Completa
**Fecha:** Febrero 2026
**Objetivo:** Crear las 6 tablas relacionales y cargar todos los datos

### Conceptos SQL de hoy:
- `CREATE TABLE` — crear una tabla con sus columnas y tipos de datos
- `INSERT INTO` — agregar filas a una tabla
- `FOREIGN KEY` — relacionar tablas entre sí
- `JOIN` — cruzar datos de dos tablas

In [11]:
# ============================================================
# SESIÓN 2: BASE DE DATOS SQL COMPLETA
# ============================================================
# En la Sesión 1 usamos :memory: — los datos vivían en RAM
# Hoy creamos un archivo .db real que persiste
# ============================================================

import sqlite3
import pandas as pd

# Crear conexión a archivo real
conn = sqlite3.connect('waste_management.db')
cursor = conn.cursor()

print("✅ Base de datos 'waste_management.db' creada")
print("   Esta base de datos se guarda como archivo")
print("   La encontrarás en el panel de archivos de Colab 📁")

✅ Base de datos 'waste_management.db' creada
   Esta base de datos se guarda como archivo
   La encontrarás en el panel de archivos de Colab 📁


In [12]:
# ============================================================
# CREAR LAS 6 TABLAS
# ============================================================
# CREATE TABLE = definir la estructura de una tabla
# Es como diseñar las columnas de una hoja de Excel
# pero con tipos de datos estrictos
#
# Tipos de datos en SQLite:
#   INTEGER  → números enteros (1, 2, 100)
#   REAL     → números decimales (9.3, 415.0)
#   TEXT     → texto ("México", "Europa")
#   PRIMARY KEY → identificador único de cada fila
#   REFERENCES  → relación con otra tabla (FOREIGN KEY)
# ============================================================

# Eliminar tablas si ya existen (para empezar limpio)
cursor.executescript("""
    DROP TABLE IF EXISTS indicadores_ec;
    DROP TABLE IF EXISTS composicion_residuos;
    DROP TABLE IF EXISTS disposicion_residuos;
    DROP TABLE IF EXISTS generacion_residuos;
    DROP TABLE IF EXISTS empresa_referencia;
    DROP TABLE IF EXISTS paises;
""")

# TABLA 1: Países
cursor.execute("""
CREATE TABLE paises (
    pais_id          INTEGER PRIMARY KEY,
    codigo_iso       TEXT NOT NULL,
    nombre           TEXT NOT NULL,
    region           TEXT NOT NULL,
    subregion        TEXT,
    poblacion_m      REAL,
    pib_per_capita   REAL
)
""")

# TABLA 2: Generación de residuos por año
cursor.execute("""
CREATE TABLE generacion_residuos (
    gen_id           INTEGER PRIMARY KEY,
    pais_id          INTEGER REFERENCES paises(pais_id),
    anio             INTEGER NOT NULL,
    total_kt         REAL,
    kg_per_capita    REAL,
    fuente           TEXT
)
""")

# TABLA 3: Disposición final
cursor.execute("""
CREATE TABLE disposicion_residuos (
    disp_id          INTEGER PRIMARY KEY,
    pais_id          INTEGER REFERENCES paises(pais_id),
    anio             INTEGER NOT NULL,
    operacion        TEXT NOT NULL,
    porcentaje       REAL,
    fuente           TEXT
)
""")

# TABLA 4: Composición de residuos
cursor.execute("""
CREATE TABLE composicion_residuos (
    comp_id          INTEGER PRIMARY KEY,
    pais_id          INTEGER REFERENCES paises(pais_id),
    anio             INTEGER NOT NULL,
    tipo_residuo     TEXT NOT NULL,
    porcentaje       REAL,
    fuente           TEXT
)
""")

# TABLA 5: Indicadores de economía circular
cursor.execute("""
CREATE TABLE indicadores_ec (
    ind_id                 INTEGER PRIMARY KEY,
    pais_id                INTEGER REFERENCES paises(pais_id),
    anio                   INTEGER NOT NULL,
    tasa_reciclaje_pct     REAL,
    tasa_compostaje_pct    REAL,
    tasa_recuperacion_pct  REAL,
    meta_2030_pct          REAL,
    fuente                 TEXT
)
""")

# TABLA 6: Empresa de referencia (Capa 3)
cursor.execute("""
CREATE TABLE empresa_referencia (
    emp_id                  INTEGER PRIMARY KEY,
    nombre                  TEXT,
    estado                  TEXT,
    tipo_residuo_principal  TEXT,
    volumen_anual_t         REAL,
    porcentaje_reciclado    REAL,
    certificaciones         TEXT,
    anio_dato               INTEGER
)
""")

conn.commit()
print("✅ 6 tablas creadas exitosamente:")
print("   1. paises")
print("   2. generacion_residuos")
print("   3. disposicion_residuos")
print("   4. composicion_residuos")
print("   5. indicadores_ec")
print("   6. empresa_referencia")

✅ 6 tablas creadas exitosamente:
   1. paises
   2. generacion_residuos
   3. disposicion_residuos
   4. composicion_residuos
   5. indicadores_ec
   6. empresa_referencia


✅ Datos cargados en todas las tablas:
   paises:               19 registros
   generacion_residuos:  19 registros
   disposicion_residuos: 76 registros
   indicadores_ec:       19 registros
   empresa_referencia:   1 registro


In [15]:
# ============================================================
# TU PRIMER JOIN
# ============================================================
# Sin JOIN solo puedes ver pais_id (número), no el nombre
# Con JOIN cruzas las tablas y ves nombre + datos juntos
#
# INNER JOIN = solo filas que existen en AMBAS tablas
# ON = la columna que conecta las dos tablas
# ============================================================

query_join1 = """
SELECT p.nombre,
       p.region,
       g.kg_per_capita,
       g.fuente
FROM generacion_residuos g
INNER JOIN paises p ON g.pais_id = p.pais_id
ORDER BY g.kg_per_capita DESC
"""

resultado = pd.read_sql_query(query_join1, conn)
print("JOIN 1: Generación per cápita con nombre de país")
print("="*55)
print(resultado.to_string(index=False))

JOIN 1: Generación per cápita con nombre de país
      nombre         region  kg_per_capita        fuente
   Dinamarca         Europa            844      EUROSTAT
    Alemania         Europa            632      EUROSTAT
      Italia         Europa            591      EUROSTAT
     Austria         Europa            560      EUROSTAT
     Francia         Europa            541      EUROSTAT
      Grecia         Europa            533      EUROSTAT
      España         Europa            530      EUROSTAT
Países Bajos         Europa            527      EUROSTAT
    Portugal         Europa            516      EUROSTAT
     Bélgica         Europa            505      EUROSTAT
      México         México            415 SEMARNAT 2021
       Chile América Latina            395      BID 2022
      Brasil América Latina            379      BID 2022
    Bulgaria         Europa            365      EUROSTAT
   Argentina América Latina            359      BID 2022
     Polonia         Europa            

In [16]:
# ============================================================
# JOIN ANALÍTICO: México vs promedio europeo
# ============================================================
# Esta query responde la pregunta 3 del proyecto:
# ¿Cuánto residuo genera México vs Europa?

query_join3 = """
SELECT
    p.nombre,
    p.region,
    g.kg_per_capita,
    i.tasa_reciclaje_pct,
    ROUND(i.meta_2030_pct - i.tasa_reciclaje_pct, 1) AS gap_meta_2030
FROM paises p
INNER JOIN generacion_residuos g ON p.pais_id = g.pais_id
INNER JOIN indicadores_ec i ON p.pais_id = i.pais_id
WHERE p.nombre IN ('México', 'Alemania', 'España', 'Polonia', 'Brasil')
ORDER BY i.tasa_reciclaje_pct DESC
"""

resultado3 = pd.read_sql_query(query_join3, conn)
print("JOIN 3: México vs países de referencia")
print("="*65)
print(resultado3.to_string(index=False))
print("\n→ gap_meta_2030 = cuántos puntos le falta a cada país para su meta")

JOIN 3: México vs países de referencia
  nombre         region  kg_per_capita  tasa_reciclaje_pct  gap_meta_2030
Alemania         Europa            632                67.3            2.7
  España         Europa            530                35.7           34.3
 Polonia         Europa            358                35.5           34.5
  México         México            415                 9.3           20.7
  Brasil América Latina            379                 4.0           21.0

→ gap_meta_2030 = cuántos puntos le falta a cada país para su meta


In [17]:
# ============================================================
# GUARDAR Y CERRAR LA BASE DE DATOS
# ============================================================

conn.commit()
conn.close()
print("✅ Base de datos guardada: waste_management.db")
print("\n📊 RESUMEN SESIÓN 2:")
print("   • 6 tablas creadas con relaciones entre ellas")
print("   • 134 registros cargados en total")
print("   • 3 JOINs completados — cruzando hasta 3 tablas")
print("\n💡 INSIGHT CLAVE:")
print("   Polonia genera MENOS residuos que México (358 vs 415 kg)")
print("   pero recicla casi 4x más (35.5% vs 9.3%)")
print("   → Polonia es el benchmark realista para México, no Alemania")

✅ Base de datos guardada: waste_management.db

📊 RESUMEN SESIÓN 2:
   • 6 tablas creadas con relaciones entre ellas
   • 134 registros cargados en total
   • 3 JOINs completados — cruzando hasta 3 tablas

💡 INSIGHT CLAVE:
   Polonia genera MENOS residuos que México (358 vs 415 kg)
   pero recicla casi 4x más (35.5% vs 9.3%)
   → Polonia es el benchmark realista para México, no Alemania
